# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
import json
import nltk
import re
from collections import Counter
from nltk.corpus import stopwords
from itertools import combinations
from nltk.stem import WordNetLemmatizer

# Configuration

In [2]:
# Configura la ubicación del controlador de Chrome
chrome_driver_path = "chromedriver-win64/chromedriver.exe"

# Configura las opciones del navegador para desactivar las notificaciones
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
# chrome_options.add_argument("--headless")

# Functions

In [10]:
import pandas as pd

def save_common_words_to_excel(common_words, searchElement, country):
    excel_name = searchElement + "-" + country + ".xlsx"
    if common_words:
        # Crear un DataFrame a partir de las palabras más comunes
        df = pd.DataFrame(common_words, columns=['Palabra', 'Frecuencia'])

        # Calcular el total de frecuencias
        total_frecuencias = df['Frecuencia'].sum()

        # Calcular el porcentaje de aparición con dos decimales
        df['Porcentaje'] = ((df['Frecuencia'] / total_frecuencias) * 100).round(2)

        # Guardar el DataFrame en un archivo Excel
        df.to_excel(excel_name, index=False)
        print(f"Palabras comunes guardadas en {excel_name}")
    else:
        print("No hay datos para guardar en el archivo Excel.")

# Scrapper

In [9]:
# Configura la ubicación del controlador de Chrome
chrome_driver_path = "chromedriver-win64/chromedriver.exe"

# Configura las opciones del navegador para desactivar las notificaciones
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
# chrome_options.add_argument("--headless")

def SEOBusqueda(searchElement,country):
    # Inicializa el servicio del controlador y el navegador
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    url = "https://www.amazon." + country + "/"
    driver.get(url)

    # Espera explícitamente hasta que el campo de búsqueda esté presente en la página
    wait = WebDriverWait(driver, 20)
    try:
        search_input = wait.until(EC.presence_of_element_located((By.ID, "twotabsearchtextbox")))
    except:
        try:
            search_input = wait.until(EC.presence_of_element_located((By.ID, "nav-bb-search")))
        except:
            print("No se pudo encontrar un campo de búsqueda válido.")

    search_input.send_keys(searchElement)
    search_input.send_keys(Keys.RETURN)

    # Espera a que la página de resultados cargue
    wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="a-size-base-plus a-color-base a-text-normal"]')))

    # Verifica si hay un captcha en la página
    captcha_present = False
    try:
        captcha_element = driver.find_element(By.XPATH, '//div[@class="a-box a-alert a-alert-info a-spacing-base"]')
        captcha_present = captcha_element.is_displayed()
    except:
        pass

    if captcha_present:
        print("Se detectó un captcha. Por favor, completa el captcha manualmente y luego presiona Enter aquí.")
        input("Presiona Enter para continuar después de completar el captcha.")

    # Continúa con el resto del código después de que se haya completado el captcha
    spans = driver.find_elements(By.XPATH, '//span[@class="a-size-base-plus a-color-base a-text-normal"]')


    # Inicializa una lista para almacenar todas las palabras y combinaciones de dos palabras
    todas_las_palabras = []

    # Obtiene la lista de palabras de paro en español
    stop_words = set(stopwords.words('spanish'))

    # Agrega las preposiciones y determinantes que deseas filtrar
    stop_words.update(['el', 'la', 'los', 'las', 'un', 'una', 'de', 'en', 'con', 'para', 'por', 'a', 'su', 'sus', 'mi', 'mis', 'tu', 'tus', 'nuestro', 'nuestra', 'nuestros', 'nuestras'])  # Agrega más según sea necesario

    # Inicializa el lematizador de palabras
    lemmatizer = WordNetLemmatizer()

    # Itera a través de los elementos encontrados y agrega sus textos a la lista
    for span in spans:
        texto = span.text
        palabras = texto.split()  # Tokeniza el texto en palabras
        
        # Normaliza las palabras (lematización)
        palabras_normalizadas = [lemmatizer.lemmatize(palabra) for palabra in palabras]
        
        # Limpia las palabras de comas y caracteres especiales
        palabras_limpias = [palabra.replace(',', '') for palabra in palabras_normalizadas]

        # Filtra las palabras limpias que no están en la lista de palabras de paro, no son números ni son palabras de una letra
        palabras_filtradas = [palabra for palabra in palabras_limpias if palabra.lower() not in stop_words and not palabra.isdigit() and len(palabra) > 1]

        todas_las_palabras.extend(palabras_filtradas)
    
    # Crea un diccionario que cuente la frecuencia de cada palabra o combinación
    contador = Counter(todas_las_palabras)

    # Ordena el diccionario por frecuencia de mayor a menor
    palabras_mas_comunes = contador.most_common()

    # Imprime las palabras más comunes
    # for palabra, frecuencia in palabras_mas_comunes:
    #     print(f'{palabra}: {frecuencia}')

    # Cierra el navegador
    driver.quit()

    save_common_words_to_excel(palabras_mas_comunes, searchElement, country)


In [11]:
SEOBusqueda("Nappe", "fr")

Palabras comunes guardadas en Nappe-fr.xlsx
